In [ ]:
# -*- coding: utf-8 -*-
import os, warnings, joblib, json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import RidgeCV

# Environmental configuration to prevent threading conflicts in notebooks
os.environ.setdefault("OMP_NUM_THREADS", "1")
os.environ.setdefault("OPENBLAS_NUM_THREADS", "1")
os.environ.setdefault("MKL_NUM_THREADS", "1")

warnings.filterwarnings("ignore")

# ================== Paths & Configuration ==================
CSV_PATH = "9_makale_data_guncel.csv"
TARGET   = "band_gap"
ID_COLS  = ["material_id", "formula_pretty"]

# Model Filenames
PATH_CAT  = "catboost.cbm"
PATH_LGB  = "lightgbm_model.pkl"
PATH_XGB  = "xgb_model.pkl"
PATH_RF   = "rf_model.joblib"
PATH_MLP  = "mlp_trained.joblib"
PATH_SCLR = "mlp_scaler.joblib"

RANDOM_STATE = 42

# ================== Utility Functions ==================
def report(tag, y_true, y_pred):
    """Calculates and prints regression performance metrics."""
    r2  = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    print(f"{tag:>10} -> R²={r2:.6f} | MAE={mae:.6f} | MSE={mse:.6f}")

# ================== Data Preprocessing ==================
df = pd.read_csv(CSV_PATH)
df = df.drop(columns=[c for c in ID_COLS if c in df.columns], errors="ignore")
y  = df[TARGET].astype(float)
X  = df.drop(columns=[TARGET]).select_dtypes(include=[np.number]).copy()

# Stratified Splitting (70/15/15)
# Using quantiles to ensure balanced distribution of band gaps in all sets
y_bins = pd.qcut(y, q=10, duplicates="drop", labels=False)
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.15, shuffle=True, random_state=RANDOM_STATE, stratify=y_bins
)

y_bins_tv = pd.qcut(y_trainval, q=10, duplicates="drop", labels=False)
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.17647, shuffle=True, 
    random_state=RANDOM_STATE, stratify=y_bins_tv
)

# ================== Model Loading ==================
base_models = {}

# Load available models with error handling
try:
    from catboost import CatBoostRegressor, Pool
    if os.path.exists(PATH_CAT):
        cat = CatBoostRegressor()
        cat.load_model(PATH_CAT)
        base_models["catboost"] = cat
except: pass

if os.path.exists(PATH_LGB): base_models["lightgbm"] = joblib.load(PATH_LGB)
if os.path.exists(PATH_XGB): base_models["xgboost"] = joblib.load(PATH_XGB)
if os.path.exists(PATH_RF):  base_models["random_forest"] = joblib.load(PATH_RF)

# MLP requires its specific scaler
if os.path.exists(PATH_MLP) and os.path.exists(PATH_SCLR):
    base_models["mlp"] = (joblib.load(PATH_MLP), joblib.load(PATH_SCLR))

print(f"Active Models: {list(base_models.keys())}")

# ================== Stacking Process ==================



def try_predict_one(name, X_df):
    """Wrapper to handle model-specific prediction logic."""
    try:
        if name == "catboost":
            X_cat = X_df.copy()
            X_cat.columns = [c.replace(" ", "_") for c in X_cat.columns]
            return base_models[name].predict(Pool(X_cat)).ravel()
        elif name == "mlp":
            m, s = base_models[name]
            return m.predict(s.transform(X_df)).ravel()
        else:
            return base_models[name].predict(X_df).ravel()
    except Exception as e:
        print(f"Error predicting with {name}: {e}")
        return None

def generate_meta_features(X_df, model_names):
    """Stacks predictions into a feature matrix for the meta-learner."""
    preds = {name: try_predict_one(name, X_df) for name in model_names}
    return pd.DataFrame({k: v for k, v in preds.items() if v is not None}, index=X_df.index)

# Generate predictions for Meta-Learner training (Validation set) and Final testing
Z_val  = generate_meta_features(X_val, base_models.keys())
Z_test = generate_meta_features(X_test, base_models.keys())

# Meta-Learner: Ridge Regression with Built-in Cross-Validation for Alpha

meta_learner = RidgeCV(alphas=np.logspace(-6, 3, 30))
meta_learner.fit(Z_val, y_val)

# Final Performance Results
print("\n=== FINAL ENSEMBLE RESULTS ===")
report("Validation", y_val, meta_learner.predict(Z_val))
report("Test", y_test, meta_learner.predict(Z_test))

# Weight Analysis
weights = pd.Series(meta_learner.coef_, index=Z_val.columns).sort_values(ascending=False)
print("\nModel Weights in Ensemble:")
print(weights)

In [ ]:
# -*- coding: utf-8 -*-
import os, warnings, joblib, json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import RidgeCV

# Environmental configuration to prevent threading conflicts in notebooks
os.environ.setdefault("OMP_NUM_THREADS", "1")
os.environ.setdefault("OPENBLAS_NUM_THREADS", "1")
os.environ.setdefault("MKL_NUM_THREADS", "1")

warnings.filterwarnings("ignore")

# ================== Paths & Configuration ==================
CSV_PATH = "9_makale_data_guncel.csv"
TARGET   = "band_gap"
ID_COLS  = ["material_id", "formula_pretty"]

# Model Filenames
PATH_CAT  = "catboost.cbm"
PATH_LGB  = "lightgbm_model.pkl"
PATH_XGB  = "xgb_model.pkl"
PATH_RF   = "rf_model.joblib"
PATH_MLP  = "mlp_trained.joblib"
PATH_SCLR = "mlp_scaler.joblib"

RANDOM_STATE = 42

# ================== Utility Functions ==================
def report(tag, y_true, y_pred):
    """Calculates and prints regression performance metrics."""
    r2  = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    print(f"{tag:>10} -> R²={r2:.6f} | MAE={mae:.6f} | MSE={mse:.6f}")

# ================== Data Preprocessing ==================
df = pd.read_csv(CSV_PATH)
df = df.drop(columns=[c for c in ID_COLS if c in df.columns], errors="ignore")
y  = df[TARGET].astype(float)
X  = df.drop(columns=[TARGET]).select_dtypes(include=[np.number]).copy()

# Stratified Splitting (70/15/15)
# Using quantiles to ensure balanced distribution of band gaps in all sets
y_bins = pd.qcut(y, q=10, duplicates="drop", labels=False)
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.15, shuffle=True, random_state=RANDOM_STATE, stratify=y_bins
)

y_bins_tv = pd.qcut(y_trainval, q=10, duplicates="drop", labels=False)
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.17647, shuffle=True, 
    random_state=RANDOM_STATE, stratify=y_bins_tv
)

# ================== Model Loading ==================
base_models = {}

# Load available models with error handling
try:
    from catboost import CatBoostRegressor, Pool
    if os.path.exists(PATH_CAT):
        cat = CatBoostRegressor()
        cat.load_model(PATH_CAT)
        base_models["catboost"] = cat
except: pass

if os.path.exists(PATH_LGB): base_models["lightgbm"] = joblib.load(PATH_LGB)
if os.path.exists(PATH_XGB): base_models["xgboost"] = joblib.load(PATH_XGB)
if os.path.exists(PATH_RF):  base_models["random_forest"] = joblib.load(PATH_RF)

# MLP requires its specific scaler
if os.path.exists(PATH_MLP) and os.path.exists(PATH_SCLR):
    base_models["mlp"] = (joblib.load(PATH_MLP), joblib.load(PATH_SCLR))

print(f"Active Models: {list(base_models.keys())}")

# ================== Stacking Process ==================



def try_predict_one(name, X_df):
    """Wrapper to handle model-specific prediction logic."""
    try:
        if name == "catboost":
            X_cat = X_df.copy()
            X_cat.columns = [c.replace(" ", "_") for c in X_cat.columns]
            return base_models[name].predict(Pool(X_cat)).ravel()
        elif name == "mlp":
            m, s = base_models[name]
            return m.predict(s.transform(X_df)).ravel()
        else:
            return base_models[name].predict(X_df).ravel()
    except Exception as e:
        print(f"Error predicting with {name}: {e}")
        return None

def generate_meta_features(X_df, model_names):
    """Stacks predictions into a feature matrix for the meta-learner."""
    preds = {name: try_predict_one(name, X_df) for name in model_names}
    return pd.DataFrame({k: v for k, v in preds.items() if v is not None}, index=X_df.index)

# Generate predictions for Meta-Learner training (Validation set) and Final testing
Z_val  = generate_meta_features(X_val, base_models.keys())
Z_test = generate_meta_features(X_test, base_models.keys())

# Meta-Learner: Ridge Regression with Built-in Cross-Validation for Alpha

meta_learner = RidgeCV(alphas=np.logspace(-6, 3, 30))
meta_learner.fit(Z_val, y_val)

# Final Performance Results
print("\n=== FINAL ENSEMBLE RESULTS ===")
report("Validation", y_val, meta_learner.predict(Z_val))
report("Test", y_test, meta_learner.predict(Z_test))

# Weight Analysis
weights = pd.Series(meta_learner.coef_, index=Z_val.columns).sort_values(ascending=False)
print("\nModel Weights in Ensemble:")
print(weights)

In [ ]:
# -*- coding: utf-8 -*-
import os, warnings, joblib, pickle, numpy as np, pandas as pd
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import RidgeCV
from itertools import combinations

warnings.filterwarnings("ignore")

# --- Paths & Configuration ---
CSV_PATH = "9_makale_data_guncel.csv"
TARGET = "band_gap"
ID_COLS = ["material_id", "formula_pretty"]

PATH_CAT = "catboost.cbm"
PATH_LGB = "lightgbm_model.pkl"
PATH_XGB = "xgb_model.pkl"
PATH_RF  = "rf_model.joblib"
PATH_MLP = "mlp_artifacts/mlp_trained.joblib"
PATH_SCLR = "mlp_artifacts/mlp_scaler.joblib"

# ================== XGBoost Wrapper ==================
import xgboost as xgb
class XGBSklearnWrapper:
    """Standardizes XGBoost prediction calls for Sklearn compatibility."""
    def __init__(self, booster, feature_names):
        self.booster = booster
        self.feature_names_ = list(feature_names)
    def predict(self, X):
        if isinstance(X, pd.DataFrame):
            X = X[self.feature_names_]
        dm = xgb.DMatrix(X)
        return self.booster.predict(dm, iteration_range=(0, self.booster.best_iteration + 1))

# ================== Data Loading ==================
df = pd.read_csv(CSV_PATH).drop(columns=[c for c in ID_COLS if c in df.columns], errors="ignore")
y_all = df[TARGET].astype(float)
X_all = df.drop(columns=[TARGET]).select_dtypes(include="number").copy()

# Load predefined splits
idx_tr = pd.read_csv("split_train.csv")["idx"].values
idx_va = pd.read_csv("split_val.csv")["idx"].values
idx_te = pd.read_csv("split_test.csv")["idx"].values

X_train, y_train = X_all.iloc[idx_tr], y_all.iloc[idx_tr]
X_val,   y_val   = X_all.iloc[idx_va], y_all.iloc[idx_va]
X_test,  y_test  = X_all.iloc[idx_te], y_all.iloc[idx_te]

print(f"Dataset Split: Train={len(idx_tr)} | Val={len(idx_va)} | Test={len(idx_te)}")

# ================== Loading Base Models ==================
base_models = {}

# CatBoost
try:
    from catboost import CatBoostRegressor, Pool
    if os.path.exists(PATH_CAT):
        m = CatBoostRegressor(); m.load_model(PATH_CAT); base_models["catboost"] = m
except Exception as e:
    print("[catboost] Error loading:", e)

# LightGBM / RF
if os.path.exists(PATH_LGB): base_models["lightgbm"] = joblib.load(PATH_LGB)
if os.path.exists(PATH_RF):  base_models["random_forest"] = joblib.load(PATH_RF)

# XGBoost
if os.path.exists(PATH_XGB):
    try:
        with open(PATH_XGB, "rb") as f: base_models["xgboost"] = pickle.load(f)
    except Exception as e:
        print("[xgboost] Error loading:", e)

# MLP
mlp_scaler = joblib.load(PATH_SCLR) if os.path.exists(PATH_SCLR) else None
mlp_model  = joblib.load(PATH_MLP)  if os.path.exists(PATH_MLP)  else None
if mlp_model is not None and mlp_scaler is not None:
    base_models["mlp"] = (mlp_model, mlp_scaler)

if not base_models: raise RuntimeError("No base models found.")
print("Loaded models:", list(base_models.keys()))

# ================== Prediction Helper ==================
def try_predict(name, X):
    """Handles model-specific input formatting for predictions."""
    try:
        if name == "catboost":
            X_cat = X.copy()
            X_cat.columns = [c.replace(" ", "_") for c in X_cat.columns]
            return np.asarray(base_models["catboost"].predict(Pool(X_cat))).ravel()
        elif name == "mlp":
            mlp, scaler = base_models["mlp"]
            return np.asarray(mlp.predict(scaler.transform(X))).ravel()
        else:
            return np.asarray(base_models[name].predict(X)).ravel()
    except Exception as e:
        print(f"[{name}] Prediction failed: {e}")
        return None

# ================== Active Models Check ==================
active = []
for n in list(base_models.keys()):
    p = try_predict(n, X_val.head(256))
    if p is not None and np.isfinite(p).all(): active.append(n)
print("Active models available for testing:", active)

# ================== Meta-Feature Matrix Helper ==================
def preds_matrix(X, names):
    """Stacks predictions to create meta-features."""
    cols, arr = [], []
    for n in names:
        p = try_predict(n, X)
        if p is not None: cols.append(n); arr.append(p)
    return pd.DataFrame(np.vstack(arr).T, columns=cols, index=X.index)

# ================== Combinatorial Search ==================
best_combo = None
best_r2 = -1
results = []

print("\nStarting combination search...")
for r in range(1, len(active) + 1):
    for combo in combinations(active, r):
        # Generate meta-features for the current subset
        Z_val  = preds_matrix(X_val,  combo)
        Z_test = preds_matrix(X_test, combo)

        # Fit Ridge meta-learner on validation predictions
        meta = RidgeCV(alphas=np.logspace(-6, 3, 30))
        meta.fit(Z_val, y_val)

        # Evaluate on test predictions
        y_pred_test = meta.predict(Z_test)
        r2 = r2_score(y_test, y_pred_test)

        results.append((combo, r2))
        print(f"Combo {combo} -> Test R²: {r2:.6f}")

        if r2 > best_r2:
            best_r2 = r2
            best_combo = combo

print("\n=== OPTIMIZATION SUMMARY ===")
print(f"Best Combination Found: {best_combo}")
print(f"Best Test R² Achieved: {best_r2:.6f}")

In [ ]:
# ================== Final Predictions with Best Combination ==================
print("\n=== FINAL PREDICTIONS & ANALYSIS ===")

# Generate meta-features using the best combination of models identified previously
Z_val  = preds_matrix(X_val,  best_combo)
Z_test = preds_matrix(X_test, best_combo)

# Final Meta-Learner (RidgeCV) fit on the validation predictions
meta = RidgeCV(alphas=np.logspace(-6, 3, 30))
meta.fit(Z_val, y_val)

# Predict for both sets
y_pred_val  = meta.predict(Z_val)
y_pred_test = meta.predict(Z_test)

# ================== Comprehensive Error Analysis ==================
import numpy as np

# Absolute errors for the validation set
errors = np.abs(y_val - y_pred_val)

# 1) Success Rate by Error Thresholds
print("\n--- Success Rate by Error Thresholds ---")
thresholds = [0.25, 0.5, 1.0] # error in eV
for t in thresholds:
    percentage = np.mean(errors < t) * 100
    print(f"{percentage:.2f}% of model predictions have an error below {t} eV.")

# 2) Percentile-Based Error Distribution
print("\n--- Percentile-Based Error Distribution ---")
for p in [50, 75, 90]:
    th = np.percentile(errors, p)
    print(f"{p}% of predictions have an error below {th:.3f} eV.")

# 3) Analysis for a Specific Range (Example: 5.5–6.0 eV)
print("\n--- Range Analysis (5.5–6.0 eV) ---")
mask = (y_pred_val >= 5.5) & (y_pred_val <= 6.0)
subset_errors = np.abs(y_val[mask] - y_pred_val[mask])

if len(subset_errors) > 0:
    print(f"Total number of predictions in this range: {len(subset_errors)}")
    for p in [50, 75, 90]:
        th = np.percentile(subset_errors, p)
        print(f"{p}% of predictions in this range have an error below {th:.3f} eV.")
else:
    print("No predictions found in this specific range.")

# 4) Signed Error Breakdown (Bias Detection in 0–4 eV range)
print("\n--- Signed Error Report (0–4 eV Range) ---")
def error_analysis(y_true, y_pred, min_val=0, max_val=4):
    """Calculates directional bias (over vs. under prediction)."""
    mask = (y_pred >= min_val) & (y_pred <= max_val)
    y_true_sub = y_true[mask]
    y_pred_sub = y_pred[mask]
    diffs = y_pred_sub - y_true_sub # Positive = Over-predicting, Negative = Under-predicting

    pos_mask = diffs > 0
    neg_mask = diffs < 0

    return {
        "total_predictions": len(diffs),
        "positive_count": np.sum(pos_mask),
        "negative_count": np.sum(neg_mask),
        "positive_mean_error": np.mean(diffs[pos_mask]) if np.any(pos_mask) else 0,
        "negative_mean_error": np.mean(diffs[neg_mask]) if np.any(neg_mask) else 0,
        "overall_mean_error": np.mean(diffs) if len(diffs) > 0 else 0,
    }

report = error_analysis(y_val, y_pred_val, 0, 4)
for key, value in report.items():
    print(f"{key}: {value}")

In [ ]:
# ================== Final Prediction with Best Combination ==================
print("\n=== FINAL PREDICTIONS & ANALYSIS ===")

# Create meta-feature matrices using the optimal model subset
Z_val  = preds_matrix(X_val,  best_combo)
Z_test = preds_matrix(X_test, best_combo)

# Final Meta-Learner (RidgeCV) trained on validation predictions
meta = RidgeCV(alphas=np.logspace(-6, 3, 30))
meta.fit(Z_val, y_val)

# Generate final predictions
y_pred_val  = meta.predict(Z_val)
y_pred_test = meta.predict(Z_test)

# ================== Error Analysis ==================
import numpy as np

# Absolute error calculation for the validation set
errors = np.abs(y_val - y_pred_val)

# 1) Success Rate based on Error Thresholds
print("\n--- Success Rate by Error Thresholds ---")
thresholds = [0.25, 0.5, 1.0] # error values in eV
for t in thresholds:
    percentage = np.mean(errors < t) * 100
    print(f"{percentage:.2f}% of predictions have an error below {t} eV.")

# 2) Percentile-Based Error Distribution
# Useful for understanding the typical error magnitude (MAE is just the average)
print("\n--- Percentile-Based Error Distribution ---")
for p in [50, 75, 90]:
    th = np.percentile(errors, p)
    print(f"{p}% of predictions have an error below {th:.3f} eV.")
    
# 3) Range-Specific Analysis Function
def range_analysis(y_true, y_pred, low, high):
    """Analyzes model performance within a specific band gap window."""
    print(f"\n--- {low}–{high} eV Range Analysis ---")
    mask = (y_pred >= low) & (y_pred <= high)
    subset_errors = np.abs(y_true[mask] - y_pred[mask])
    
    print(f"Total predictions in this range: {len(subset_errors)}")
    if len(subset_errors) > 0:
        for p in [50, 75, 90]:
            th = np.percentile(subset_errors, p)
            print(f"{p}% of predictions in this range have an error below {th:.3f} eV.")
    else:
        print("No predictions found within this range.")

# Deep analysis for high-gap insulators (4.5–6.0 eV)
range_analysis(y_val, y_pred_val, 4.5, 6.0)

# Deep analysis for metals and semiconductors (0.0–4.5 eV)
range_analysis(y_val, y_pred_val, 0.0, 4.5)

# 4) Signed Error Analysis (Bias Detection in 0–4 eV range)
print("\n--- Signed Error Report (0–4 eV Range) ---")
def bias_analysis(y_true, y_pred, min_val=0, max_val=4):
    """Identifies if the model systematically over-predicts or under-predicts."""
    mask = (y_pred >= min_val) & (y_pred <= max_val)
    y_true_sub = y_true[mask]
    y_pred_sub = y_pred[mask]
    
    # Positive result means over-prediction, negative means under-prediction
    diffs = y_pred_sub - y_true_sub

    pos_mask = diffs > 0
    neg_mask = diffs < 0

    return {
        "total_predictions": len(diffs),
        "over_prediction_count": np.sum(pos_mask),
        "under_prediction_count": np.sum(neg_mask),
        "mean_over_prediction_error": np.mean(diffs[pos_mask]) if np.any(pos_mask) else 0,
        "mean_under_prediction_error": np.mean(diffs[neg_mask]) if np.any(neg_mask) else 0,
        "overall_bias": np.mean(diffs) if len(diffs) > 0 else 0,
    }

report = bias_analysis(y_val, y_pred_val, 0, 4)
for key, value in report.items():
    print(f"{key}: {value}")

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error

def interval_error_analysis(y_true, y_pred, intervals):
    """Calculates MAE and RMSE for specific prediction ranges."""
    results = []
    for low, high in intervals:
        # Filter data points within the current interval
        mask = (y_pred >= low) & (y_pred < high)
        y_t, y_p = y_true[mask], y_pred[mask]
        
        if len(y_t) > 0:
            mae  = mean_absolute_error(y_t, y_p)
            mse  = mean_squared_error(y_t, y_p)
            rmse = np.sqrt(mse)
            results.append({
                "Interval": f"{low}–{high} eV",
                "Count": len(y_t),
                "MAE": mae,
                "RMSE": rmse
            })
        else:
            results.append({
                "Interval": f"{low}–{high} eV",
                "Count": 0,
                "MAE": None,
                "RMSE": None
            })
    return results

# Define custom intervals (e.g., metals/semiconductors vs. insulators)
test_intervals = [(0, 2), (2, 4), (0, 4.5), (4.5, 6)]

# Execute analysis on validation set
analysis_data = interval_error_analysis(y_val, y_pred_val, test_intervals)

# Display results as a DataFrame
df_results = pd.DataFrame(analysis_data)
print(df_results)